### 卷积神经网络（LeNet）
> - 卷积神经网络就是含卷积层的网络
> - LeNet:一个早期用来识别手写数字图像的卷积神经网络
> - LeNet展示了通过梯度下降训练卷积神经网络可以达到手写数字识别在当时最先进的结果，第一次将卷积神经网络推上舞台，为世人所知

In [2]:
import tensorflow as tf
print(tf.__version__)

2.13.0


#### 1 LeNet模型
- LeNet分为卷积层块和全连接层块两个部分
- 卷积层块里的基本单位是卷积层后接最大池化层：卷积层用来识别图像里的空间模式，如线条和物体局部，之后的最大池化层则用来降低卷积层对位置的敏感性
- 卷积层块的输出形状为(批量大小, 通道, 高, 宽)
- 当卷积层块的输出传入全连接层块时，全连接层块会将小批量中每个样本变平（flatten）

In [5]:
# 通过Sequential类来实现LeNet模型
net = tf.keras.models.Sequential([
    # 卷积层块里的基本单位是卷积层后接最大池化层
    # 卷积层块由两个这样 MaxPool2D的基本单位重复堆叠构成。
    # 每个卷积层都使用5×5的窗口，在输出上使用sigmoid激活函数
    # 第一个卷积层输出通道数为6
    tf.keras.layers.Conv2D(filters=6,kernel_size=5,activation='sigmoid',input_shape=(28,28,1)),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    # 第二个卷积层输出通道数则增加到16，因为第二个卷积层比第一个卷积层的输入的高和宽要小
    tf.keras.layers.Conv2D(filters=16,kernel_size=5,activation='sigmoid'),
    # 卷积层块的两个最大池化层的窗口形状均为2×2，且步幅为2
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    # 当卷积层块的输出传入全连接层块时，全连接层块会将小批量中每个样本变平（flatten）
    tf.keras.layers.Flatten(),
    # 全连接层块含3个全连接层。它们的输出个数分别是120、84和10，其中10为输出的类别个数。
    tf.keras.layers.Dense(120,activation='sigmoid'),
    tf.keras.layers.Dense(84,activation='sigmoid'),
    tf.keras.layers.Dense(10,activation='sigmoid')
])

In [6]:
# 构造一个高和宽均为28的单通道数据样本
X = tf.random.uniform((1,28,28,1))
for layer in net.layers:
    X = layer(X)
    print(layer.name, 'output shape\t', X.shape)


conv2d_4 output shape	 (1, 24, 24, 6)
max_pooling2d_4 output shape	 (1, 12, 12, 6)
conv2d_5 output shape	 (1, 8, 8, 16)
max_pooling2d_5 output shape	 (1, 4, 4, 16)
flatten_2 output shape	 (1, 256)
dense_6 output shape	 (1, 120)
dense_7 output shape	 (1, 84)
dense_8 output shape	 (1, 10)


#### 2 获取数据和训练模型

In [7]:
# 使用Fashion-MNIST作为训练数据集
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()


4422102/4422102 [==============================] - 1s 0us/step


In [8]:
train_images = tf.reshape(train_images, (train_images.shape[0],train_images.shape[1],train_images.shape[2], 1))
print(train_images.shape)

test_images = tf.reshape(test_images, (test_images.shape[0],test_images.shape[1],test_images.shape[2], 1))


(60000, 28, 28, 1)


In [9]:
# 损失函数和训练算法依然采用交叉熵损失函数(cross entropy)和小批量随机梯度下降(SGD)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.9, momentum=0.0, nesterov=False)

net.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [10]:
net.fit(train_images, train_labels, epochs=5, validation_split=0.1)

Epoch 1/5
1688/1688 [==============================] - 11s 6ms/step - loss: 1.0518 - accuracy: 0.5861 - val_loss: 0.5873 - val_accuracy: 0.7740
Epoch 2/5
1688/1688 [==============================] - 11s 6ms/step - loss: 0.6111 - accuracy: 0.7557 - val_loss: 0.5833 - val_accuracy: 0.7623
Epoch 3/5
1688/1688 [==============================] - 10s 6ms/step - loss: 0.5289 - accuracy: 0.7907 - val_loss: 0.5115 - val_accuracy: 0.8022
Epoch 4/5
1688/1688 [==============================] - 10s 6ms/step - loss: 0.4876 - accuracy: 0.8117 - val_loss: 0.5164 - val_accuracy: 0.8003
Epoch 5/5
1688/1688 [==============================] - 10s 6ms/step - loss: 0.4686 - accuracy: 0.8183 - val_loss: 0.4680 - val_accuracy: 0.8277


In [ ]:
net.evaluate(test_images, test_labels, verbose=2)
